In [19]:
import json

from tqdm.auto import tqdm
import pandas as pd
import nltk
import requests

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


tqdm.pandas()


client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [20]:
from nltk.tokenize import word_tokenize

text = "Лемматизированная форма слова листья это лист"
tokens = word_tokenize(text)
print(tokens)

['Лемматизированная', 'форма', 'слова', 'листья', 'это', 'лист']


In [21]:
df = pd.read_csv("with_names.csv")

In [22]:
df["tags"] = df["text"].apply(word_tokenize)

In [23]:
import re

def clean_from_non_alpha(words: list[str]) -> list[str]:

    cleaned_list = []
    for i in words:
        cleaned_word = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9]', '', i)
        if cleaned_word:
            cleaned_list.append(cleaned_word)

    return cleaned_list

In [24]:
df["tags"] = df["tags"].apply(clea_from_non_alpha)

In [25]:
def construct_lemma_prompt(text: str) -> str:

    prompt = f"""
        Приведи слово "{text}" в первоначальную форму.
        В качестве ответа дай только это слово в первоначальной форме.
        Если слово не было предоставлено, то в качестве овтета дай ""
        """
    return prompt

In [26]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [27]:
def get_tags(text: str) -> str:

    prompt = construct_lemma_prompt(text)
    tags_from_llm = request_from_llm(prompt)
    return tags_from_llm

In [28]:
def get_tags_from_list(tags: list[str]) -> list[str]:

    res_list = []
    for tag in tags:
        res_list.append(get_tags(tag))
    return res_list

In [29]:
df["tags"] = df["tags"].progress_apply(get_tags)(get_tags_from_list)

  0%|          | 0/1265 [00:00<?, ?it/s]

TypeError: 'Series' object is not callable

In [30]:
df["tags"]

0                                            [ПРИЛОЖЕНИЕ]
1                            [к, Закону, СанктПетербурга]
2                                          [О, Стратегии]
3                               [социальноэкономического]
4                             [развития, СанктПетербурга]
                              ...                        
1260    [улучшение, инвестиционного, климата, СанктПет...
1261    [Исходя, из, оценки, ретроспективных, данных, ...
1262    [Реализация, Стратегии, 2035, осуществляется, ...
1263    [Государственные, программы, СанктПетербурга, ...
1264    [Перечень, государственных, программ, СанктПет...
Name: tags, Length: 1265, dtype: object